In [1]:
!pip install wandb -q
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_tanish")

wandb.login(key = secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanish-jain140301 (tanish1403) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
!python --version

Python 3.11.13


In [3]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 -q
# !pip install mmcv-full==1.7.1 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html -q
!pip install mmdet==2.28.2 -q
!pip install -U openmim -q
!mim install "mmengine>=0.7.0" -q
!pip install xmltodict -q  # For dataset conversion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 381.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.2 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
    

In [4]:
!mim install 'mmcv==2.0.1'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 9.1 MB/s eta 0:00:00


In [5]:
!mim install 'mmcv-full'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 9.0 MB/s eta 0:00:00


In [6]:
# Step 6: Prepare Dataset (SCCOS to DOTA format)
import os
import shutil
import random
import xmltodict
import tqdm
import math

dataset_path = "/kaggle/input/sccos-dataset/"
working_dir = "/kaggle/working/sccos_dota"
train_images_dir = os.path.join(working_dir, "train/images")
train_labels_dir = os.path.join(working_dir, "train/labels")
val_images_dir = os.path.join(working_dir, "val/images")
val_labels_dir = os.path.join(working_dir, "val/labels")
test_images_dir = os.path.join(working_dir, "test/images")
test_labels_dir = os.path.join(working_dir, "test/labels")

In [7]:
# Step 4: Clone and Install MMRotate 0.3.4
!git clone https://github.com/zhangpeng2001/nirnet.git /kaggle/working/mmrotate
%cd /kaggle/working/mmrotate
# !git checkout v0.3.4  # Ensure exact version
!pip install -r requirements/build.txt -q
!pip install -v -e . -q

Cloning into '/kaggle/working/mmrotate'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 482 (delta 143), reused 474 (delta 135), pack-reused 0 (from 0)
Receiving objects: 100% (482/482), 11.52 MiB | 36.64 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/kaggle/working/mmrotate
Obtaining file:///kaggle/working/mmrotate
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 7.4 MB/s eta 0:00:00
  Running setup.py develop for mmrotate


In [8]:
# Step 5: Verify Installations
!pip list | grep -E 'torch|mmcv|mmdet|mmengine|mmrotate'

mmcv                                  2.0.1
mmcv-full                             1.7.2
mmdet                                 2.28.2
mmengine                              0.10.7
mmrotate                              0.3.4               /kaggle/working/mmrotate
pytorch-ignite                        0.5.2
pytorch-lightning                     2.5.2
torch                                 2.0.1+cu118
torchao                               0.10.0
torchaudio                            2.0.2+cu118
torchdata                             0.11.0
torchinfo                             1.8.0
torchmetrics                          1.7.3
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.15.2+cu118


In [9]:
%%writefile configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_sccos.py
_base_ = [
    '../_base_/datasets/dotav1.py', '../_base_/schedules/schedule_1x.py',
    '../_base_/default_runtime.py'
]

angle_version = 'le90'
model = dict(
    type='OrientedRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='OrientedRPNHead',
        in_channels=256,
        feat_channels=256,
        version=angle_version,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='MidpointOffsetCoder',
            angle_range=angle_version,
            target_means=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0, 0.5, 0.5]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(
            type='SmoothL1Loss', beta=0.1111111111111111, loss_weight=1.0)),
    roi_head=dict(
        type='OrientedStandardRoIHead',
        bbox_roi_extractor=dict(
            type='RotatedSingleRoIExtractor',
            roi_layer=dict(
                type='RoIAlignRotated',
                out_size=7,
                sample_num=2,
                clockwise=True),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='RotatedShared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=15,
            bbox_coder=dict(
                type='DeltaXYWHAOBBoxCoder',
                angle_range=angle_version,
                norm_factor=None,
                edge_swap=True,
                proj_xy=True,
                target_means=(.0, .0, .0, .0, .0),
                target_stds=(0.1, 0.1, 0.2, 0.2, 0.1)),
            reg_class_agnostic=True,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=0,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.8),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=False,
                iou_calculator=dict(type='RBboxOverlaps2D'),
                ignore_iof_thr=-1),
            sampler=dict(
                type='RRandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.8),
            min_bbox_size=0),
        rcnn=dict(
            nms_pre=2000,
            min_bbox_size=0,
            score_thr=0.05,
            nms=dict(iou_thr=0.1),
            max_per_img=2000)))

dataset_type = 'DOTADataset'
# data_root = '/kaggle/working/sccos_dota/'
data_root = '/kaggle/input/sccos-dota/sccos_dota/'
classes = ('ship',)  # Explicitly define classes
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024)),
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le135'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='RResize'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]
data = dict(
    samples_per_gpu=4,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train/labels/',
        img_prefix=data_root + 'train/images/',
        pipeline=train_pipeline,
        version='le135',
        classes=('ship',)),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'val/labels/',
        img_prefix=data_root + 'val/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test/labels/',
        img_prefix=data_root + 'test/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)))
evaluation = dict(interval=1, metric='mAP', save_best='mAP') # Add save_best here
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=1000,
    warmup_ratio=0.3333333333333333,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=25)
checkpoint_config = dict(interval=1) # Keep interval for saving
log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook',
             init_kwargs=dict(
                 project='SOTA',
                 name='ORCNN_train'),
             interval=100)
    ])

Writing configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_sccos.py


In [10]:
%cd /kaggle/working/mmrotate
!python tools/analysis_tools/get_flops.py configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_sccos.py

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
OrientedRCNN(
  41.14 M, 100.000% Params, 211.431 GFLOPs, 100.000% FLOPs, 
  (backbone): ResNet(
    23.283 M, 56.594% Params, 86.095 GFLOPs, 40.720% FLOPs, 
    (conv1): Conv2d(0.0 M, 0.000% Params, 2.466 GFLOPs, 1.166% FLOPs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3

In [11]:
# Step 8: Train the Model
%cd /kaggle/working/mmrotate
!mkdir -p /kaggle/working/runs/ORCNN_train


!python tools/train.py \
    configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_sccos.py \
    --work-dir /kaggle/working/runs/ORCNN_train \
    --gpus 1 \


print("Training completed.")

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your 

In [12]:
# Test FIRNet (uncomment to run)
import os
import glob

%cd /kaggle/working/mmrotate

# Step 1: Find the best checkpoint file automatically
checkpoint_dir = '/kaggle/working/runs/ORCNN_train'
best_checkpoint_path = None
# Search for the best checkpoint file
for filename in glob.glob(os.path.join(checkpoint_dir, 'best_mAP_*.pth')):
    best_checkpoint_path = filename
    break  # Take the first one found

if best_checkpoint_path:
    print(f"Found best checkpoint: {best_checkpoint_path}")
    
    # Step 2: Run the test command with the found checkpoint
    !mkdir -p /kaggle/working/runs/ORCNN_test
    !python tools/test.py \
        configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_sccos.py \
        {best_checkpoint_path} \
        --eval mAP \
        --out /kaggle/working/runs/ORCNN_test/results.pkl \
        --show-dir /kaggle/working/runs/ORCNN_test/vis
    
    print("FIRNet testing completed. Results saved.")
else:
    print(f"Error: No best checkpoint found in {checkpoint_dir}. Please ensure training was successful.")

/kaggle/working/mmrotate
Error: No best checkpoint found in /kaggle/working/runs/ORCNN_train. Please ensure training was successful.
